# 04. 리뷰 감성 분석 (Sentiment Analysis)

## 📌 이 노트북의 목적
- 리뷰 텍스트를 읽고 긍정적인지 부정적인지 자동으로 판단하는 머신러닝 모델을 만듭니다
- **분류(Classification)** 문제: 텍스트를 특정 카테고리로 분류합니다 (추천 vs 미추천)
- 자연어 처리(NLP)의 기본을 배웁니다

## 🎯 학습 목표
1. 분류 문제의 이해
2. 텍스트 데이터 전처리 방법
3. 텍스트를 숫자로 변환하는 방법 (벡터화)
4. 여러 분류 모델 비교하기
5. 혼동 행렬로 성능 분석하기

## 💡 실생활 응용
- 고객 리뷰 자동 분류 (긍정/부정)
- 소셜 미디어 감성 분석
- 스팸 메일 필터링
- 뉴스 기사 카테고리 분류

---

## 1. 분류(Classification)란?

### 📖 개념 설명

**분류**는 데이터를 미리 정해진 카테고리(클래스)로 나누는 머신러닝 기법입니다.

### 회귀 vs 분류 비교:

| 구분 | 회귀(Regression) | 분류(Classification) |
|------|------------------|----------------------|
| 출력 | 연속된 숫자 | 카테고리 (범주) |
| 예시 | 3.5점, 4.2점 | 긍정/부정, O/X |
| 문제 | "평점이 몇 점일까?" | "긍정일까 부정일까?" |

### 우리가 풀 문제:
- **입력(Input)**: 리뷰 텍스트 ("This product is amazing!")
- **출력(Output)**: 추천 여부 (추천 or 미추천)
- **목표**: 리뷰 텍스트만 보고 사용자가 제품을 추천하는지 예측하기

### 이진 분류(Binary Classification):
- 2개의 클래스로 나누는 문제
- 우리 문제: 추천(1) vs 미추천(0)
- 다른 예시: 스팸(1) vs 정상(0), 합격(1) vs 불합격(0)

---

## 2. 라이브러리 임포트

In [ ]:
# ============================================================
# 2-1. 기본 라이브러리
# ============================================================

# 데이터 처리
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 텍스트 처리
import re  # 정규표현식 (Regular Expression) - 패턴 매칭
import string  # 문자열 상수 (구두점, 공백 등)

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

# 그래프 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 기본 라이브러리 임포트 완료")

In [ ]:
# ============================================================
# 2-2. 자연어 처리(NLP) 라이브러리
# ============================================================
# NLTK: Natural Language Toolkit
# 텍스트 데이터를 처리하는 파이썬 라이브러리

import nltk

# NLTK 데이터 다운로드
# stopwords: 의미 없는 단어 목록 (a, the, is, are 등)
# punkt: 문장을 단어로 나누는 토크나이저
print("📥 NLTK 데이터 다운로드 중...")
print("   (처음 실행 시 시간이 걸릴 수 있습니다)\n")

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

# 불용어(Stopwords) 가져오기
from nltk.corpus import stopwords
# 불용어: 'the', 'is', 'at', 'a' 같이 의미가 없는 흔한 단어들
# 이런 단어들은 분석에서 제거하는 것이 좋음

from nltk.tokenize import word_tokenize
# 토크나이저: 문장을 단어로 나누는 도구
# 예: "I love this!" → ["I", "love", "this", "!"]

# 영어 불용어 목록
stop_words = set(stopwords.words('english'))

print("✅ NLTK 라이브러리 준비 완료")
print(f"   불용어 개수: {len(stop_words)}개")
print(f"   예시: {list(stop_words)[:10]}")

In [ ]:
# ============================================================
# 2-3. 머신러닝 라이브러리
# ============================================================

# 데이터 분할
from sklearn.model_selection import train_test_split

# 텍스트 벡터화 (텍스트 → 숫자)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# CountVectorizer: 단어의 출현 빈도수를 세는 방법
# 예: "I love love this" → {"I":1, "love":2, "this":1}

# TfidfVectorizer: TF-IDF (Term Frequency - Inverse Document Frequency)
# 단어의 빈도와 중요도를 함께 고려
# 자주 나오지만 여러 문서에 공통으로 나오는 단어는 중요도가 낮음

# 분류 모델들
from sklearn.linear_model import LogisticRegression
# 로지스틱 회귀: 분류를 위한 선형 모델 (회귀라는 이름이지만 분류 모델!)
# 결과를 확률로 변환 (0~1 사이)

from sklearn.naive_bayes import MultinomialNB
# 나이브 베이즈: 베이즈 정리를 이용한 확률 기반 분류 모델
# 텍스트 분류에 매우 효과적
# '나이브(순진한)'라는 이름: 단어들이 독립적이라고 가정 (실제로는 아니지만)

from sklearn.ensemble import RandomForestClassifier
# 랜덤 포레스트 분류기: 여러 결정 트리를 사용

from sklearn.svm import LinearSVC
# SVM (Support Vector Machine): 데이터를 구분하는 최적의 경계선을 찾는 모델
# LinearSVC: 선형 커널을 사용하는 빠른 버전

# 평가 지표
from sklearn.metrics import (
    accuracy_score,      # 정확도: 전체 중 맞춘 비율
    precision_score,     # 정밀도: 긍정으로 예측한 것 중 실제 긍정 비율
    recall_score,        # 재현율: 실제 긍정 중 긍정으로 예측한 비율
    f1_score,           # F1: 정밀도와 재현율의 조화평균
    classification_report,  # 종합 리포트
    confusion_matrix     # 혼동 행렬: 예측 결과를 표로 정리
)

print("✅ 머신러닝 라이브러리 임포트 완료")
print("\n📚 준비된 모델:")
print("  1. 로지스틱 회귀 (Logistic Regression) - 빠르고 해석하기 쉬움")
print("  2. 나이브 베이즈 (Naive Bayes) - 텍스트 분류에 특화")
print("  3. 랜덤 포레스트 (Random Forest) - 복잡한 패턴 학습")
print("  4. SVM (Support Vector Machine) - 고성능 분류")

---
## 3. 데이터 로드 및 준비

In [ ]:
# ============================================================
# 3-1. 전처리된 리뷰 데이터 로드
# ============================================================

print("📂 전처리된 리뷰 데이터를 불러오는 중...")
print("   (데이터가 많아서 시간이 걸릴 수 있습니다)\n")

# CSV 파일 읽기
# nrows를 지정하면 처음 n개 행만 읽음 (빠른 테스트용)
# 전체 데이터를 사용하려면 nrows 제거
reviews = pd.read_csv('cleaned_reviews.csv')

print(f"✅ 데이터 로드 완료: {reviews.shape[0]:,}개 리뷰, {reviews.shape[1]}개 컬럼")

# 데이터 미리보기
print("\n📊 데이터 미리보기 (처음 3개):")
print("="*80)
reviews[['review_text', 'rating', 'is_recommended']].head(3)

In [ ]:
# ============================================================
# 3-2. 필요한 컬럼만 선택
# ============================================================
# 감성 분석에 필요한 것: 리뷰 텍스트와 추천 여부

print("🎯 감성 분석에 필요한 데이터 준비...\n")

# review_text와 is_recommended만 선택
# 결측값이 있는 행 제거
data = reviews[['review_text', 'is_recommended']].dropna()

print(f"✅ 선택 완료: {len(data):,}개 리뷰")

# 추천 여부 분포 확인
print("\n📊 클래스 분포 (추천 여부):")
print("="*80)
class_dist = data['is_recommended'].value_counts()
print(class_dist)
print()
for label, count in class_dist.items():
    percentage = (count / len(data)) * 100
    label_name = "추천" if label else "미추천"
    print(f"{label_name}: {count:,}개 ({percentage:.1f}%)")

In [ ]:
# ============================================================
# 3-3. 클래스 불균형 확인 및 처리
# ============================================================
# 클래스 불균형: 한 클래스의 데이터가 너무 많거나 적은 경우
# 예: 긍정 90%, 부정 10% → 모델이 항상 긍정으로 예측해도 90% 정확도!

print("⚖️ 클래스 불균형 확인...\n")

# 불균형 비율 계산
positive_count = (data['is_recommended'] == True).sum()
negative_count = (data['is_recommended'] == False).sum()
imbalance_ratio = max(positive_count, negative_count) / min(positive_count, negative_count)

print(f"불균형 비율: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 3:
    print("\n⚠️ 심한 클래스 불균형이 감지되었습니다!")
    print("   해결 방법:")
    print("   1. 언더샘플링: 많은 클래스의 데이터를 줄임")
    print("   2. 오버샘플링: 적은 클래스의 데이터를 늘림")
    print("   3. 가중치 부여: 적은 클래스에 더 높은 가중치")
    print("\n   → 우리는 가중치 부여 방식을 사용하겠습니다.")
else:
    print("\n✅ 클래스 불균형이 심하지 않습니다.")

# 클래스 분포 시각화
plt.figure(figsize=(10, 6))
labels = ['미추천', '추천']
counts = [negative_count, positive_count]
colors = ['#ff6b6b', '#51cf66']

bars = plt.bar(labels, counts, color=colors, alpha=0.7, edgecolor='black')

# 막대 위에 숫자와 비율 표시
for bar, count in zip(bars, counts):
    height = bar.get_height()
    percentage = (count / len(data)) * 100
    plt.text(bar.get_x() + bar.get_width()/2, height + len(data)*0.01,
            f'{count:,}\n({percentage:.1f}%)',
            ha='center', fontsize=12, fontweight='bold')

plt.title('클래스 분포 (추천 여부)', fontsize=14, fontweight='bold')
plt.ylabel('리뷰 수', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---
## 4. 텍스트 전처리

### 왜 텍스트 전처리가 필요할까?

원본 텍스트를 그대로 사용하면 문제가 생깁니다:
- "Good", "good", "GOOD" → 같은 단어지만 다르게 인식됨
- "!!!", "..." → 의미 없는 특수문자
- "the", "is", "a" → 너무 흔해서 의미 없음

### 전처리 단계:
1. **소문자 변환**: Good → good
2. **특수문자 제거**: "amazing!!!" → "amazing"
3. **숫자 제거**: "2024" → ""
4. **불용어 제거**: "the product is good" → "product good"
5. **토큰화**: "I love this" → ["I", "love", "this"]

### 주의:
- 너무 많이 전처리하면 의미 있는 정보도 사라질 수 있습니다
- 적절한 균형이 중요합니다

In [ ]:
# ============================================================
# 4-1. 텍스트 전처리 함수 정의
# ============================================================

def preprocess_text(text):
    """
    텍스트를 전처리하는 함수
    
    Parameters:
    -----------
    text : str
        원본 텍스트
    
    Returns:
    --------
    cleaned_text : str
        전처리된 텍스트
    """
    
    # 1. 소문자 변환
    # "AMAZING" → "amazing"
    text = text.lower()
    
    # 2. 특수문자 제거 (알파벳과 공백만 남김)
    # re.sub(패턴, 대체할_문자, 원본_문자열)
    # [^a-z\s]: 알파벳(a-z)과 공백(\s)이 아닌 모든 것
    # ^는 '아닌'을 의미
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. 토큰화 (문장을 단어로 나누기)
    # "I love this" → ["I", "love", "this"]
    tokens = text.split()
    
    # 4. 불용어 제거
    # 'the', 'is', 'a' 같은 의미 없는 단어 제거
    # [단어 for 단어 in 리스트 if 조건]: 리스트 컴프리헨션
    # 조건을 만족하는 단어만 새 리스트에 포함
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. 너무 짧은 단어 제거 (2글자 이하)
    # 'a', 'I' 같은 단어 제거
    tokens = [word for word in tokens if len(word) > 2]
    
    # 6. 단어들을 다시 문장으로 합치기
    # ["love", "this"] → "love this"
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# 함수 테스트
print("🧪 텍스트 전처리 함수 테스트:\n")

test_texts = [
    "I LOVE this product!!! It's AMAZING 😊",
    "This is the WORST product ever... Don't buy it!!!",
    "Good quality for the price. I recommend it 100%!"
]

for i, text in enumerate(test_texts, 1):
    cleaned = preprocess_text(text)
    print(f"예시 {i}:")
    print(f"  원본: {text}")
    print(f"  전처리 후: {cleaned}")
    print()

print("✅ 전처리 함수 정의 완료")

In [ ]:
# ============================================================
# 4-2. 전체 데이터에 전처리 적용
# ============================================================

print("🔄 전체 리뷰 데이터 전처리 중...")
print("   (시간이 걸릴 수 있습니다. 잠시만 기다려주세요)\n")

# tqdm 라이브러리가 있으면 진행바 표시
try:
    from tqdm import tqdm
    # .apply()에 progress_apply() 사용 (진행바 표시)
    tqdm.pandas(desc="전처리 진행")
    data['cleaned_text'] = data['review_text'].progress_apply(preprocess_text)
except ImportError:
    # tqdm이 없으면 일반 apply 사용
    data['cleaned_text'] = data['review_text'].apply(preprocess_text)

print("\n✅ 전처리 완료!\n")

# 전처리 결과 확인
print("📊 전처리 결과 비교 (랜덤 3개):")
print("="*80)

# 무작위로 3개 선택
sample_indices = np.random.choice(len(data), size=3, replace=False)

for i, idx in enumerate(sample_indices, 1):
    original = data.iloc[idx]['review_text']
    cleaned = data.iloc[idx]['cleaned_text']
    label = "추천" if data.iloc[idx]['is_recommended'] else "미추천"
    
    print(f"\n예시 {i} [{label}]:")
    print(f"  원본 ({len(original)}자):")
    print(f"    {original[:200]}...")  # 처음 200자만
    print(f"  전처리 후 ({len(cleaned)}자):")
    print(f"    {cleaned[:200]}...")  # 처음 200자만

# 빈 텍스트 제거
# 전처리 후 텍스트가 비어있는 경우 제거
data = data[data['cleaned_text'].str.len() > 0]

print(f"\n\n📊 최종 데이터 크기: {len(data):,}개 리뷰")

---
## 5. 텍스트 벡터화 (Text Vectorization)

### 왜 벡터화가 필요할까?

머신러닝 모델은 **숫자만** 이해할 수 있습니다.  
텍스트를 숫자 벡터로 변환해야 합니다.

### 두 가지 방법:

#### 1. CountVectorizer (단어 빈도수)
- 각 단어가 몇 번 나왔는지 세기
- 예시:
  ```
  문장: "I love love this product"
  결과: {"I":1, "love":2, "this":1, "product":1}
  벡터: [1, 2, 1, 1]
  ```

#### 2. TF-IDF (단어 중요도)
- TF (Term Frequency): 문서 내 단어 빈도
- IDF (Inverse Document Frequency): 전체 문서에서 희귀한 정도
- 공식: TF-IDF = TF × IDF
- 자주 나오지만 여러 문서에 공통으로 나오는 단어는 중요도 낮음
- 예: "good"은 모든 리뷰에 나옴 → 중요도 낮음
- 예: "fragrance"는 특정 제품 리뷰에만 나옴 → 중요도 높음

### 어떤 것을 사용할까?
- CountVectorizer: 간단하고 빠름, 이해하기 쉬움
- TF-IDF: 더 정교함, 일반적으로 성능이 더 좋음

우리는 **TF-IDF**를 사용하겠습니다!

In [ ]:
# ============================================================
# 5-1. 학습/테스트 데이터 분할
# ============================================================
# 벡터화하기 전에 먼저 데이터를 나눕니다
# 왜? 테스트 데이터의 정보가 학습에 유출되는 것을 방지

print("✂️ 데이터를 학습용(80%)과 테스트용(20%)으로 나누는 중...\n")

# X: 전처리된 텍스트
X = data['cleaned_text']

# y: 추천 여부 (True/False)
y = data['is_recommended']

# 데이터 분할
# stratify=y: 클래스 비율을 유지하면서 분할
# 예: 전체에서 긍정 80%면, 학습/테스트 모두 긍정 80%로 유지
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # 클래스 비율 유지
)

print("📊 데이터 분할 결과:")
print("="*80)
print(f"전체 데이터: {len(X):,}개")
print()
print(f"학습 데이터:")
print(f"  - 크기: {len(X_train):,}개 ({len(X_train)/len(X)*100:.1f}%)")
print(f"  - 추천: {y_train.sum():,}개 ({y_train.sum()/len(y_train)*100:.1f}%)")
print(f"  - 미추천: {(~y_train).sum():,}개 ({(~y_train).sum()/len(y_train)*100:.1f}%)")
print()
print(f"테스트 데이터:")
print(f"  - 크기: {len(X_test):,}개 ({len(X_test)/len(X)*100:.1f}%)")
print(f"  - 추천: {y_test.sum():,}개 ({y_test.sum()/len(y_test)*100:.1f}%)")
print(f"  - 미추천: {(~y_test).sum():,}개 ({(~y_test).sum()/len(y_test)*100:.1f}%)")

print("\n✅ stratify 덕분에 클래스 비율이 유지되었습니다!")

In [ ]:
# ============================================================
# 5-2. TF-IDF 벡터화
# ============================================================

print("🔢 TF-IDF로 텍스트를 숫자 벡터로 변환하는 중...\n")

# TfidfVectorizer 생성
vectorizer = TfidfVectorizer(
    max_features=5000,  # 가장 중요한 5000개 단어만 사용
    # 왜 제한? 모든 단어를 사용하면 벡터가 너무 커져서 느려짐
    
    min_df=5,  # 최소 5개 문서에 나타나야 함
    # 너무 희귀한 단어는 제외 (오타일 가능성)
    
    max_df=0.8,  # 전체 문서의 80% 이상에 나타나는 단어는 제외
    # 너무 흔한 단어는 의미가 없음
    
    ngram_range=(1, 2)  # 1개 단어(unigram)와 2개 단어(bigram)
    # 예: "not good" → "not", "good", "not good" 모두 고려
)

# 학습 데이터로 벡터라이저 학습
# .fit(): 어떤 단어들이 있는지 학습, 각 단어에 인덱스 부여
print("1. 학습 데이터로 벡터라이저 학습 중...")
vectorizer.fit(X_train)
print("   ✅ 완료")

# 학습 데이터 변환
# .transform(): 텍스트를 숫자 벡터로 변환
print("\n2. 학습 데이터 변환 중...")
X_train_tfidf = vectorizer.transform(X_train)
print(f"   ✅ 완료: {X_train_tfidf.shape}")
print(f"   → {X_train_tfidf.shape[0]:,}개 문서, {X_train_tfidf.shape[1]:,}개 특성(단어)")

# 테스트 데이터 변환 (같은 벡터라이저 사용!)
print("\n3. 테스트 데이터 변환 중...")
X_test_tfidf = vectorizer.transform(X_test)
print(f"   ✅ 완료: {X_test_tfidf.shape}")

print("\n" + "="*80)
print("✅ 벡터화 완료!")
print("="*80)

# 추출된 단어(특성) 확인
feature_names = vectorizer.get_feature_names_out()
print(f"\n📊 추출된 특성(단어) 수: {len(feature_names):,}개")
print(f"\n처음 20개 단어:")
print(list(feature_names[:20]))
print(f"\n마지막 20개 단어:")
print(list(feature_names[-20:]))

In [ ]:
# ============================================================
# 5-3. 벡터화 결과 확인
# ============================================================

print("🔍 벡터화 결과 예시\n")
print("="*80)

# 첫 번째 리뷰 확인
sample_idx = 0
sample_text = X_train.iloc[sample_idx]
sample_vector = X_train_tfidf[sample_idx]

print(f"원본 텍스트:")
print(f"  {sample_text}")
print()

# 벡터에서 0이 아닌 값들만 추출 (중요한 단어들)
# .nonzero()[1]: 0이 아닌 값의 인덱스들
nonzero_indices = sample_vector.nonzero()[1]

print(f"TF-IDF 벡터 (0이 아닌 값만):")
print(f"  총 {len(nonzero_indices)}개 단어")
print()

# 상위 10개 중요 단어 출력
word_scores = [(feature_names[i], sample_vector[0, i]) for i in nonzero_indices]
# TF-IDF 점수가 높은 순으로 정렬
word_scores.sort(key=lambda x: x[1], reverse=True)

print("가장 중요한 10개 단어 (TF-IDF 점수):")
for i, (word, score) in enumerate(word_scores[:10], 1):
    print(f"  {i}. {word}: {score:.4f}")

print("\n💡 해석:")
print("   - 점수가 높을수록 이 리뷰에서 중요한 단어입니다")
print("   - 0인 단어는 이 리뷰에 나타나지 않은 단어입니다")
print("   - 대부분의 값이 0입니다 (희소 행렬, Sparse Matrix)")

---
## 6. 모델 학습 및 평가

### 사용할 모델들:

#### 1. 로지스틱 회귀 (Logistic Regression)
- **원리**: 선형 모델 + 시그모이드 함수로 확률 계산
- **장점**: 빠르고 해석하기 쉬움
- **단점**: 복잡한 패턴은 학습 못함

#### 2. 나이브 베이즈 (Naive Bayes)
- **원리**: 베이즈 정리를 이용한 확률 계산
- **장점**: 텍스트 분류에 매우 효과적, 빠름
- **단점**: 단어들이 독립적이라고 가정 (실제로는 아님)

#### 3. 랜덤 포레스트 (Random Forest)
- **원리**: 여러 결정 트리의 투표
- **장점**: 복잡한 패턴 학습 가능
- **단점**: 느림, 많은 메모리 사용

#### 4. SVM (Support Vector Machine)
- **원리**: 최적의 결정 경계 찾기
- **장점**: 고차원 데이터에 효과적
- **단점**: 큰 데이터셋에서는 느림

### 평가 지표:

#### 혼동 행렬 (Confusion Matrix):
```
                예측: 미추천    예측: 추천
실제: 미추천    TN (맞음)     FP (틀림)
실제: 추천      FN (틀림)     TP (맞음)
```

#### 지표들:
- **정확도 (Accuracy)**: (TP + TN) / 전체  
  → 전체 중 맞춘 비율

- **정밀도 (Precision)**: TP / (TP + FP)  
  → 추천으로 예측한 것 중 실제 추천 비율  
  → 높으면: 추천으로 예측하면 정말 추천일 확률이 높음

- **재현율 (Recall)**: TP / (TP + FN)  
  → 실제 추천 중 추천으로 예측한 비율  
  → 높으면: 추천을 잘 찾아냄

- **F1 Score**: 2 × (Precision × Recall) / (Precision + Recall)  
  → 정밀도와 재현율의 조화평균  
  → 불균형 데이터에서 더 신뢰할 수 있음

In [ ]:
# ============================================================
# 6-1. 모델 평가 함수 정의
# ============================================================

def evaluate_classifier(model, X_train, X_test, y_train, y_test, model_name):
    """
    분류 모델을 학습하고 평가하는 함수
    """
    
    print(f"\n{'='*80}")
    print(f"🤖 {model_name} 학습 중...")
    print(f"{'='*80}")
    
    # 모델 학습
    model.fit(X_train, y_train)
    print("✅ 학습 완료!\n")
    
    # 예측
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # 평가 지표 계산
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    
    # 결과 출력
    print("📊 학습 데이터 성능:")
    print(f"   정확도: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
    
    print("\n📊 테스트 데이터 성능: (진짜 성능!)")
    print(f"   정확도 (Accuracy)  : {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    print(f"   정밀도 (Precision) : {test_precision:.4f}")
    print(f"   재현율 (Recall)    : {test_recall:.4f}")
    print(f"   F1 Score          : {test_f1:.4f}")
    
    # 과적합 체크
    if train_accuracy - test_accuracy > 0.05:
        print("\n⚠️ 과적합 가능성: 학습 데이터 성능이 테스트보다 높습니다.")
    else:
        print("\n✅ 과적합 없음: 학습과 테스트 성능이 비슷합니다.")
    
    # 혼동 행렬
    cm = confusion_matrix(y_test, y_test_pred)
    
    # 혼동 행렬 시각화
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['미추천', '추천'],
                yticklabels=['미추천', '추천'])
    plt.title(f'혼동 행렬 - {model_name}', fontsize=14, fontweight='bold')
    plt.ylabel('실제', fontsize=12)
    plt.xlabel('예측', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    # 혼동 행렬 설명
    tn, fp, fn, tp = cm.ravel()
    print("\n📊 혼동 행렬 해석:")
    print(f"   TN (참 부정): {tn:,}개 - 미추천을 미추천으로 맞춤 ✓")
    print(f"   FP (거짓 긍정): {fp:,}개 - 미추천을 추천으로 틀림 ✗")
    print(f"   FN (거짓 부정): {fn:,}개 - 추천을 미추천으로 틀림 ✗")
    print(f"   TP (참 긍정): {tp:,}개 - 추천을 추천으로 맞춤 ✓")
    
    # 상세 리포트
    print("\n📋 상세 분류 리포트:")
    print("="*80)
    print(classification_report(y_test, y_test_pred, 
                                target_names=['미추천', '추천']))
    
    # 결과 반환
    results = {
        'model_name': model_name,
        'model': model,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1,
        'confusion_matrix': cm
    }
    
    return results

print("✅ 평가 함수 정의 완료")

In [ ]:
# ============================================================
# 6-2. 모델 1 - 로지스틱 회귀 (Logistic Regression)
# ============================================================

print("\n" + "#"*80)
print("# 모델 1: 로지스틱 회귀 (Logistic Regression)")
print("#"*80)
print("\n📖 로지스틱 회귀란?")
print("   선형 회귀 + 시그모이드 함수를 사용하여 확률을 계산합니다.")
print("   출력이 0~1 사이의 확률로 나오며, 0.5를 기준으로 분류합니다.")
print("   빠르고 해석하기 쉬워서 베이스라인 모델로 많이 사용됩니다.")

# 로지스틱 회귀 모델 생성
# max_iter=1000: 최대 반복 횟수
# class_weight='balanced': 클래스 불균형 자동 처리
lr_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)

# 모델 학습 및 평가
lr_results = evaluate_classifier(
    model=lr_model,
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
    y_test=y_test,
    model_name="로지스틱 회귀 (Logistic Regression)"
)

In [ ]:
# ============================================================
# 6-3. 모델 2 - 나이브 베이즈 (Naive Bayes)
# ============================================================

print("\n" + "#"*80)
print("# 모델 2: 나이브 베이즈 (Naive Bayes)")
print("#"*80)
print("\n📖 나이브 베이즈란?")
print("   베이즈 정리를 이용한 확률 기반 분류 모델입니다.")
print("   각 단어가 독립적이라고 가정하여 계산합니다.")
print("   텍스트 분류에 매우 효과적이고 빠릅니다.")
print("   스팸 필터, 문서 분류 등에 널리 사용됩니다.")

# 나이브 베이즈 모델 생성
# MultinomialNB: 다항 나이브 베이즈 (단어 빈도에 적합)
nb_model = MultinomialNB()

# 모델 학습 및 평가
nb_results = evaluate_classifier(
    model=nb_model,
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
    y_test=y_test,
    model_name="나이브 베이즈 (Naive Bayes)"
)

In [ ]:
# ============================================================
# 6-4. 모델 3 - 랜덤 포레스트 (Random Forest)
# ============================================================

print("\n" + "#"*80)
print("# 모델 3: 랜덤 포레스트 (Random Forest)")
print("#"*80)
print("\n📖 랜덤 포레스트란?")
print("   여러 개의 결정 트리를 만들어서 투표로 결정합니다.")
print("   복잡한 비선형 패턴도 학습할 수 있습니다.")
print("   과적합에 강하지만 학습 시간이 오래 걸립니다.")

# 랜덤 포레스트 모델 생성
rf_model = RandomForestClassifier(
    n_estimators=100,  # 100개의 트리
    random_state=42,
    class_weight='balanced',
    n_jobs=-1  # 모든 CPU 사용
)

# 모델 학습 및 평가
rf_results = evaluate_classifier(
    model=rf_model,
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
    y_test=y_test,
    model_name="랜덤 포레스트 (Random Forest)"
)

In [ ]:
# ============================================================
# 6-5. 모델 4 - SVM (Support Vector Machine)
# ============================================================

print("\n" + "#"*80)
print("# 모델 4: SVM (Support Vector Machine)")
print("#"*80)
print("\n📖 SVM이란?")
print("   데이터를 가장 잘 구분하는 경계선(초평면)을 찾는 모델입니다.")
print("   마진(여유 공간)을 최대화하여 일반화 성능이 좋습니다.")
print("   고차원 데이터(텍스트)에 매우 효과적입니다.")

# SVM 모델 생성
# LinearSVC: 선형 커널을 사용하는 빠른 SVM
svm_model = LinearSVC(
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)

# 모델 학습 및 평가
svm_results = evaluate_classifier(
    model=svm_model,
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
    y_test=y_test,
    model_name="SVM (Support Vector Machine)"
)

---
## 7. 모델 비교 및 최종 선택

In [ ]:
# ============================================================
# 7-1. 모든 모델의 성능 비교표 생성
# ============================================================

print("\n" + "="*80)
print("📊 모든 모델 성능 비교")
print("="*80 + "\n")

# 모든 결과를 데이터프레임으로 정리
comparison = pd.DataFrame([
    {
        '모델': lr_results['model_name'],
        '정확도': lr_results['test_accuracy'],
        '정밀도': lr_results['test_precision'],
        '재현율': lr_results['test_recall'],
        'F1 Score': lr_results['test_f1']
    },
    {
        '모델': nb_results['model_name'],
        '정확도': nb_results['test_accuracy'],
        '정밀도': nb_results['test_precision'],
        '재현율': nb_results['test_recall'],
        'F1 Score': nb_results['test_f1']
    },
    {
        '모델': rf_results['model_name'],
        '정확도': rf_results['test_accuracy'],
        '정밀도': rf_results['test_precision'],
        '재현율': rf_results['test_recall'],
        'F1 Score': rf_results['test_f1']
    },
    {
        '모델': svm_results['model_name'],
        '정확도': svm_results['test_accuracy'],
        '정밀도': svm_results['test_precision'],
        '재현율': svm_results['test_recall'],
        'F1 Score': svm_results['test_f1']
    }
])

# F1 Score 기준으로 내림차순 정렬
comparison = comparison.sort_values('F1 Score', ascending=False)

print(comparison.to_string(index=False))

# 최고 성능 모델
best_model_name = comparison.iloc[0]['모델']
best_f1 = comparison.iloc[0]['F1 Score']

print("\n" + "="*80)
print(f"🏆 최고 성능 모델: {best_model_name}")
print(f"   F1 Score: {best_f1:.4f}")
print("="*80)

In [ ]:
# ============================================================
# 7-2. 모델 성능 시각화
# ============================================================

# 그래프 크기 설정
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. 정확도 비교
axes[0, 0].bar(comparison['모델'], comparison['정확도'], 
               color='skyblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('정확도 (Accuracy)', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('점수', fontsize=12)
axes[0, 0].set_ylim(0, 1)
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)
for i, (model, acc) in enumerate(zip(comparison['모델'], comparison['정확도'])):
    axes[0, 0].text(i, acc + 0.02, f'{acc:.4f}', ha='center', fontsize=10, fontweight='bold')

# 2. 정밀도 비교
axes[0, 1].bar(comparison['모델'], comparison['정밀도'], 
               color='lightcoral', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('정밀도 (Precision)', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('점수', fontsize=12)
axes[0, 1].set_ylim(0, 1)
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)
for i, (model, prec) in enumerate(zip(comparison['모델'], comparison['정밀도'])):
    axes[0, 1].text(i, prec + 0.02, f'{prec:.4f}', ha='center', fontsize=10, fontweight='bold')

# 3. 재현율 비교
axes[1, 0].bar(comparison['모델'], comparison['재현율'], 
               color='lightgreen', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('재현율 (Recall)', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('점수', fontsize=12)
axes[1, 0].set_ylim(0, 1)
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)
for i, (model, rec) in enumerate(zip(comparison['모델'], comparison['재현율'])):
    axes[1, 0].text(i, rec + 0.02, f'{rec:.4f}', ha='center', fontsize=10, fontweight='bold')

# 4. F1 Score 비교
axes[1, 1].bar(comparison['모델'], comparison['F1 Score'], 
               color='gold', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('F1 Score (종합 점수)', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('점수', fontsize=12)
axes[1, 1].set_ylim(0, 1)
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)
for i, (model, f1) in enumerate(zip(comparison['모델'], comparison['F1 Score'])):
    axes[1, 1].text(i, f1 + 0.02, f'{f1:.4f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 그래프 설명:")
print("   - 정확도: 전체 중 맞춘 비율")
print("   - 정밀도: 추천으로 예측한 것 중 실제 추천 비율")
print("   - 재현율: 실제 추천 중 추천으로 예측한 비율")
print("   - F1 Score: 정밀도와 재현율의 조화평균 (종합 지표)")

---
## 8. 실제 예측 테스트

In [ ]:
# ============================================================
# 8-1. 최고 성능 모델 선택
# ============================================================

# F1 Score가 가장 높은 모델 선택
all_results = [lr_results, nb_results, rf_results, svm_results]
best_result = max(all_results, key=lambda x: x['test_f1'])

print(f"🏆 선택된 모델: {best_result['model_name']}")
print(f"   F1 Score: {best_result['test_f1']:.4f}")

In [ ]:
# ============================================================
# 8-2. 새로운 리뷰로 예측 테스트
# ============================================================

print("\n🧪 새로운 리뷰로 감성 예측 테스트\n")
print("="*80)

# 테스트할 새로운 리뷰들
new_reviews = [
    "This product is absolutely amazing! I love it so much and highly recommend it to everyone!",
    "Terrible product. Complete waste of money. Do not buy this!",
    "It's okay, nothing special. Works as expected but not impressed.",
    "Best purchase ever! My skin has never looked better. Will definitely buy again!",
    "Horrible! Caused allergic reaction. Very disappointed and angry."
]

# 각 리뷰 예측
for i, review in enumerate(new_reviews, 1):
    # 전처리
    cleaned = preprocess_text(review)
    
    # 벡터화
    vectorized = vectorizer.transform([cleaned])
    
    # 예측
    prediction = best_result['model'].predict(vectorized)[0]
    
    # 확률 예측 (가능한 경우)
    if hasattr(best_result['model'], 'predict_proba'):
        proba = best_result['model'].predict_proba(vectorized)[0]
        confidence = proba[1] if prediction else proba[0]
    elif hasattr(best_result['model'], 'decision_function'):
        # SVM의 경우
        decision = best_result['model'].decision_function(vectorized)[0]
        # 시그모이드 함수로 확률 근사
        confidence = 1 / (1 + np.exp(-decision))
        if not prediction:
            confidence = 1 - confidence
    else:
        confidence = None
    
    # 결과 출력
    result_text = "✅ 추천" if prediction else "❌ 미추천"
    print(f"\n리뷰 {i}:")
    print(f"  원본: {review}")
    print(f"  예측: {result_text}")
    if confidence is not None:
        print(f"  확신도: {confidence:.2%}")

print("\n" + "="*80)
print("✅ 예측 테스트 완료!")

In [ ]:
# ============================================================
# 8-3. 샘플 리뷰 예측 결과 확인
# ============================================================

print("🔍 테스트 데이터에서 샘플 예측 결과 (무작위 10개):\n")
print("="*80)

# 무작위로 10개 선택
sample_indices = np.random.choice(len(X_test), size=min(10, len(X_test)), replace=False)

# 예측
y_pred = best_result['model'].predict(X_test_tfidf)

for i, idx in enumerate(sample_indices, 1):
    original_text = X_test.iloc[idx]
    actual = "추천" if y_test.iloc[idx] else "미추천"
    predicted = "추천" if y_pred[idx] else "미추천"
    correct = "✓" if y_test.iloc[idx] == y_pred[idx] else "✗"
    
    print(f"\n{i}. {correct}")
    print(f"   리뷰: {original_text[:100]}...")  # 처음 100자만
    print(f"   실제: {actual} | 예측: {predicted}")

print("\n" + "="*80)

---
## 9. 학습 요약 및 결론

In [ ]:
# ============================================================
# 9. 최종 요약
# ============================================================

print("\n" + "#"*80)
print("# 🎓 리뷰 감성 분석 - 최종 요약")
print("#"*80)

print("\n📊 프로젝트 개요:")
print("="*80)
print(f"데이터 크기: {len(data):,}개 리뷰")
print(f"학습 데이터: {len(X_train):,}개 ({len(X_train)/len(data)*100:.1f}%)")
print(f"테스트 데이터: {len(X_test):,}개 ({len(X_test)/len(data)*100:.1f}%)")
print(f"특성 차원: {X_train_tfidf.shape[1]:,}개 (TF-IDF 벡터)")

print("\n🏆 최고 성능 모델:")
print("="*80)
print(f"모델: {best_result['model_name']}")
print(f"정확도: {best_result['test_accuracy']:.4f} ({best_result['test_accuracy']*100:.2f}%)")
print(f"정밀도: {best_result['test_precision']:.4f}")
print(f"재현율: {best_result['test_recall']:.4f}")
print(f"F1 Score: {best_result['test_f1']:.4f}")

print("\n💡 배운 내용:")
print("="*80)
print("1. 분류(Classification) 문제의 이해")
print("   - 데이터를 카테고리로 나누는 머신러닝 기법")
print("   - 회귀와의 차이: 숫자 vs 카테고리")

print("\n2. 텍스트 전처리")
print("   - 소문자 변환, 특수문자 제거")
print("   - 불용어 제거 (의미 없는 단어)")
print("   - 토큰화 (문장을 단어로 나누기)")

print("\n3. 텍스트 벡터화")
print("   - TF-IDF: 단어의 빈도와 중요도를 함께 고려")
print("   - 텍스트를 숫자 벡터로 변환")
print("   - 희소 행렬 (Sparse Matrix): 대부분의 값이 0")

print("\n4. 여러 분류 모델 비교")
print("   - 로지스틱 회귀: 빠르고 해석하기 쉬움")
print("   - 나이브 베이즈: 텍스트 분류에 특화")
print("   - 랜덤 포레스트: 복잡한 패턴 학습")
print("   - SVM: 고차원 데이터에 효과적")

print("\n5. 모델 평가")
print("   - 정확도: 전체 중 맞춘 비율")
print("   - 정밀도: 긍정 예측 중 실제 긍정")
print("   - 재현율: 실제 긍정 중 긍정 예측")
print("   - F1 Score: 정밀도와 재현율의 조화평균")
print("   - 혼동 행렬: 예측 결과를 표로 정리")

print("\n" + "#"*80)
print("# 🎯 다음 단계")
print("#"*80)
print("\n다음 노트북에서 계속:")
print("  - 05_ml_recommendation_system.ipynb: 제품 추천 시스템")
print("  - 06_ml_review_helpfulness.ipynb: 리뷰 유용성 예측")

print("\n" + "="*80)
print("✅ 노트북 완료!")
print("="*80)